In [1]:
from pyspark.mllib.recommendation import ALS
import math
import re
from time import time




########################################
# Configuration.                       #
########################################

# Paths to the available data sets
DATASET_RATINGS_PATH = "./ratings_train.csv"
# A small set should be used here to calibrate the algorithm.
# Small set isn't supplied by college, it is by movielens however.
DATASET_RATINGS_SMALL_PATH = ""
DATASET_RATINGS_TEST_PATH = "./ratings_test.csv"
DATASET_MOVIES_PATH = "./movies.csv"
# A small set should be used here.
DATASET_MOVIES_SMALL_PATH = ""

# The minimum number of reviews a movie must have, to recommend it reliably
REVIEW_MIN_AMOUNT = 30

# Split ratio for the small training/calibration data
TRAIN_SMALL_SPLIT = [6, 2, 2]

# Split ratio for the complete training data
TRAIN_COMPLETE_SPLIT = [7, 3]

# Define whether to test the complete data set
TRAIN_COMPLETE_TEST = False

# Seed to use for training
TRAIN_SEED = 5

# Ranks to use for training
TRAIN_RANKS = [4, 6, 8, 10, 12]

# Number of iterations to train for
TRAIN_ITERATIONS = 10

# The number of movies to recommend for the user
RECOMMENDATION_AMOUNT = 12



########################################
# Function definitions.                #
########################################

# Define some helper functions
# TODO: Separator parameter is not doing anything right now.
def readCSV(fname, removeHeader=False, separator=','):
    print("Loading file", fname, "...")
    rdd = sc.textFile(fname)
    if removeHeader:
        firstline = rdd.first()
        rdd = rdd.filter(lambda x: x != firstline)
    return rdd.map(lambda x: re.split(r'' + separator + '(?=(?:"[^"]*?(?: [^"]*)*))|' + separator + '(?=[^"' + separator + ']+(?:' + separator + '|$))', x))

# Benchmark time
global benchmark

# Reset the benchmark
def benchmark_reset():
    global benchmark
    benchmark = time()
    
# Get the benchmarked value
def benchmark_get():
    global benchmark
    return str(round(time() - benchmark, 2)) + " s"




########################################
# Load datasets.                       #
########################################

print("Preparing to start making suggestions...")

print("Loading all dataset files...")
benchmark_reset()

# Load the ratings data
ratings_data = readCSV(DATASET_RATINGS_PATH, removeHeader=True, separator='::')
if len(DATASET_RATINGS_SMALL_PATH) > 0:
    small_ratings_data = readCSV(DATASET_RATINGS_SMALL_PATH, removeHeader=True, separator='::')
if len(DATASET_RATINGS_TEST_PATH) > 0:
    test_ratings_data = readCSV(DATASET_RATINGS_TEST_PATH, removeHeader=True, separator='::')

# Load the movies data
movies_data = readCSV(DATASET_MOVIES_PATH, removeHeader=True,separator=',')
if len(DATASET_MOVIES_SMALL_PATH) > 0:
    small_movies_data = readCSV(DATASET_MOVIES_SMALL_PATH, removeHeader=True,separator=',')

print("Datasets loaded, took", benchmark_get(), "\n")




########################################
# Parse datasets.                      #
########################################

print("Parsing datasets...")
benchmark_reset()

# Parse the complete ratings set
# [user_id, movie_id, rating, timestamp] -> (user_id, movie_id, rating)
ratings_data = ratings_data\
        .map(lambda x: (int(x[0]), int(x[1]), float(x[2])))\
        .cache()
print("There are", ratings_data.count(), "ratings in the complete dataset.")

# Parse the small ratings set
if len(DATASET_RATINGS_SMALL_PATH) > 0:
    small_ratings_data = small_ratings_data\
            .map(lambda x: (int(x[0]), int(x[1]), float(x[2])))\
            .cache()
    print("There are", small_ratings_data.count(), "ratings in the small dataset.")

# Parse the test ratings set
if len(DATASET_RATINGS_TEST_PATH) > 0:
    test_ratings_data = test_ratings_data\
            .map(lambda x: (int(x[0]), int(x[1])))\
            .cache()
    print("There are", test_ratings_data.count(), "ratings in the small dataset.")

# Parse the complete movies set
# [id, title, genres[]] -> (id, title)
movies_data = movies_data\
        .map(lambda x: (\
                        int(x[0]),\
                        x[1].replace('\"', ''),\
                        x[2].replace("(no genres listed)", "").split("|")))\
        .cache()
print("There are", movies_data.count(), "movies in the complete dataset.")
      
# Parse the small ratings set
if len(DATASET_MOVIES_SMALL_PATH) > 0:
    small_movies_data = small_movies_data\
            .map(lambda x: (int(x[0]), x[1]))\
            .cache()
    print("There are", small_movies_data.count(), "movies in the small dataset.")

print("Parsing took", benchmark_get(), "\n")




########################################
# Create users database.               #
########################################

print("Creating set of users...")
benchmark_reset()

# Get the user IDs
user_ids = ratings_data.map(lambda x: x[0]).distinct().collect()

print("Got", len(user_ids), "users, took", benchmark_get(), "\n")




########################################
# Calculate movie review count.        #
########################################

print("Calculate movie ratings amount...")
benchmark_reset()

# Helper method to count the number of ratings for each movie
def calc_avg_count(id_ratings):
    number = len(id_ratings[1])
    return id_ratings[0],\
            (number, float(sum(x for x in id_ratings[1])) / number)

# Count the number of ratings per movie
movie_id_ratings = (ratings_data\
                            .map(lambda x: tuple(x[1:3]))\
                            .groupByKey())
movie_id_ratings_avg = movie_id_ratings.map(calc_avg_count)

# Create (movie_id, review_count) tuples, filter tuples that didn't reach the review amount threshold
movie_id_ratings_count = movie_id_ratings_avg\
        .filter(lambda x: x[1][0] >= REVIEW_MIN_AMOUNT)\
        .map(lambda x: (x[0], x[1][0]))\
        .cache()

print("Calculation done, took", benchmark_get(), "\n")




########################################
# Filter movies without many ratings.  #
########################################

print("Filtering movies with less than", REVIEW_MIN_AMOUNT, "ratings as not relevant...")
benchmark_reset()

# Create a list of movie IDs that are allowed
allowed_movie_ids = movie_id_ratings_count\
        .map(lambda x: x[0])\
        .collect()
        
# Filter the movies
movies_data = movies_data\
        .filter(lambda x: x[0] in allowed_movie_ids)\
        .cache()

print("Filtered movies, took", benchmark_get(), "\n")




########################################
# Calibrate machine learning.          #
########################################

if len(DATASET_RATINGS_SMALL_PATH) > 0:
    print("Calibrating by training on small dataset...")
else:
    print("Using default calibration for data set, as no small dataset is supplied")
benchmark_reset()

# Some parameters
regularization_parameter = 0.1
errors = [0] * len(TRAIN_RANKS)
err = 0

# Create some training sets based on the small data
if len(DATASET_RATINGS_SMALL_PATH) > 0:
    training_set, validation_set, test_set = small_ratings_data\
            .randomSplit(TRAIN_SMALL_SPLIT, seed = 0)
    prediction_validation = validation_set\
            .map(lambda x: tuple(x[0:2]))
    prediction_test = test_set\
            .map(lambda x: tuple(x[0:2]))

# Rememer the minimum error value, best rank and iteration
min_error = float("inf")
best_rank = 10
best_iteration = -1

if len(DATASET_RATINGS_SMALL_PATH) > 0:
    # Loop through each rank to train
    for rank in TRAIN_RANKS:
        # Train on the selected rank
        model = ALS.train(training_set,\
                          rank,\
                          seed = TRAIN_SEED,\
                          iterations = TRAIN_ITERATIONS,\
                          lambda_ = regularization_parameter)

        # Make some predictions to test
        predictions = model\
                .predictAll(prediction_validation)\
                .map(lambda x: (tuple(x[0:2]), x[2]))
        rating_predictions = validation_set\
                .map(lambda x: ((int(x[0]), int(x[1])), float(x[2])))\
                .join(predictions)

        # Calculate the predeiction error value, and store it
        error = math.sqrt(rating_predictions\
                          .map(lambda x: (x[1][0] - x[1][1])**2)\
                          .mean())
        errors[err] = error
        err += 1

        # Update the minimum error value and the best rank
        print("Training small dataset with rank", rank, "which has RMSE:", error)
        if error < min_error:
            min_error = error
            best_rank = rank

    print("Trained small dataset calibration, took", benchmark_get(), " with a best rank:", best_rank, "\n")




########################################
# Train on complete dataset.           #
########################################

print("Training complete dataset...")
benchmark_reset()

# Randomly split the data, to use for training and testing
training_set, test_set = ratings_data.randomSplit(TRAIN_COMPLETE_SPLIT,\
                                                  seed = 0)

# Train using the data sets
trained_model = ALS.train(training_set,\
                           best_rank,\
                           seed = TRAIN_SEED,\
                           iterations = TRAIN_ITERATIONS,\
                           lambda_ = regularization_parameter)

print("Training took", benchmark_get(), "\n")




########################################
# Test trained data.                   #
########################################

if TRAIN_COMPLETE_TEST:
    print("Testing trained complete dataset...")
    benchmark_reset()

    # Create a test prediction set
    test_predict_set = test_set.map(lambda x: tuple(x[0:2]))

    # Make come predictions to test
    predictions = trained_model\
            .predictAll(test_predict_set)\
            .map(lambda x: (tuple(x[0:2]), x[2]))
    rates_predictions = test_set\
            .map(lambda x: ((int(x[0]), int(x[1])), float(x[2])))\
            .join(predictions)

    # Calculate the error value
    error = math.sqrt(\
                      rates_predictions\
                              .map(lambda x: (x[1][0] - x[1][1])**2)\
                              .mean())

    print("Trained complete dataset test, took", benchmark_get(), " with an RMSE:", error, "\n")



print("Ready to suggest!")

Preparing to start making suggestions...
Loading all dataset files...
Loading file ./ratings_train.csv ...
Loading file ./ratings_test.csv ...
Loading file ./movies.csv ...
Datasets loaded, took 1.51 s 

Parsing datasets...
There are 19861976 ratings in the complete dataset.
There are 50000 ratings in the small dataset.
There are 27278 movies in the complete dataset.
Parsing took 74.49 s 

Creating set of users...
Got 138493 users, took 13.05 s 

Calculate movie ratings amount...
Calculation done, took 0.04 s 

Filtering movies with less than 30 ratings as not relevant...
Filtered movies, took 18.91 s 

Using default calibration for data set, as no small dataset is supplied
Training complete dataset...
Training took 79.39 s 

Ready to suggest!


In [2]:
########################################
# Function definitions.                #
########################################

# Predict the ratings for all movies the user hasn't rated yet.
#
# Parameters:
# - user_id: The ID of the user to predict ratings for.
#
# Returns movie rating predictions.
def predict_movie_ratings(user_id):
    # Create a list of IDs of movies already rated by the user
    rated_ids = ratings_data\
            .filter(lambda x: x[0] == user_id)\
            .map(lambda x: x[0])\
            .collect()

    # Get all movie IDs that haven't been rated by the user
    unrated_ids = (movies_data\
                       .filter(lambda x: x[0] not in rated_ids)\
                       .map(lambda x: (user_id, x[0])))

    # Predict the recommendation value for all unrated movies for this user
    predictions = trained_model.predictAll(unrated_ids)

    # Transform the prediction result into proper tuples
    # (movie_id, predicted_rating)
    predictions = predictions.map(lambda x: (x.product, x.rating))
    
    # Saturate the list of tuples with the movie titles and number of ratings
    predictions = predictions\
            .join(movies_data)\
            .join(movies_data.map(lambda x: (x[0], x[2])))\
            .join(movie_id_ratings_count)

    # Remap the recommendations to get usable tuples:
    # (title, predicted_rating, rating_count, genres)
    return predictions\
            .map(lambda x: (x[0], x[1][0][0][1], x[1][0][0][0], x[1][0][1], x[1][1]))

        
        
# Predict the top movies to watch for the given user.
#
# Parameters:
# - user_id: The ID of the user to predict movies for.
def predict_top_movies(user_id):
    # Print a status message
    print("Predicting top movies for user", user_id, "...")
    benchmark_reset()

    # Predict ratings for unwatched movies for this user
    predictions = predict_movie_ratings(user_id)
    
    # Take the top list of movies for the user as a list
    top_movies = predictions.takeOrdered(RECOMMENDATION_AMOUNT,\
                            key = lambda x: -x[2])
    
    # Print the benchmark time, and return the top movies
    print("Top movies predicted, took", benchmark_get(), "\n")
    return top_movies




########################################
# Recommendation process.              #
########################################

# Add my personal user 9999999 in front of the list of user IDs
#user_ids = [9999999] + user_ids

i = 0

# Loop through the list of users, and get movie recommendations for each one of them
for user_id in user_ids:
#     # Increase the counter and make sure we didn't reach the limit yet
#     i += 1
#     if i > 25:
#         break

    # Collect the results
    result = predict_top_movies(user_id)
    
    # Create a list of tuples holding the user ID and movie ID
    result_ids = []
    for entry in result:
        result_ids = result_ids + [(user_id, entry[0])]
    
    # Print the result header
    print(\
          "{:<10}{:<45}{:<8}{:<9}{}"\
              .format("MOVIE ID", "MOVIE NAME", "GUESS", "RATINGS", "GENRES")\
         )
    
    # Print the result entries
    for movie_id, movie_name, rating_guess, genres, ratings in result:
        print(\
              "{:<10}{:<45}{:<8}{:<9}{}"\
                  .format(\
                        movie_id,\
                        movie_name[:43],\
                        round(rating_guess, 4),\
                        ratings,\
                        ", ".join(genres)\
                ))
    
    # Check whether any of the top rated movies is in the test list, and print the result
    hitcount = test_ratings_data.filter(lambda x: x in result_ids).count()
    if hitcount > 0:
        print("> Top 10 does have a hit in the test set!")
    else:
        print("> No hit in the test set.")
        
    # Print the footer
    print("\n")

print("...")

Predicting top movies for user 81923 ...
Top movies predicted, took 11.63 s 

MOVIE ID  MOVIE NAME                                   GUESS   RATINGS  GENRES
92500     Dylan Moran: Yeah, Yeah (2011)               3.8565  31       Comedy
92498     Dylan Moran Live: What It Is (2009)          3.7732  37       Comedy
26453     Smiley's People (1982)                       3.6815  52       Drama, Mystery
92494     Dylan Moran: Monster (2004)                  3.6553  52       Comedy, Documentary
26674     Prime Suspect (1991)                         3.6107  39       Crime, Drama, Mystery, Thriller
77177     Wild China (2008)                            3.6045  32       Documentary
105250    Century of the Self, The (2002)              3.5982  43       Documentary
116136    Olive Kitteridge (2014)                      3.5834  32       Drama
49917     When the Levees Broke: A Requiem in Four Ac  3.5614  96       Documentary
1148      Wallace & Gromit: The Wrong Trousers (1993)  3.5492  14871    